In [1]:
import os
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.metrics import mean_squared_error

from PATHS import HOME, DATASET_PATH, ANIME_DATASET_2023, DATA_PREPROCESSING

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to C:\Users\MINH
[nltk_data]     HUNG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\MINH
[nltk_data]     HUNG\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\MINH
[nltk_data]     HUNG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
print('HOME', HOME)
print('DATASET FOLDER', DATASET_PATH)
print('ANIME DATASET 2023', ANIME_DATASET_2023)
print('DATA PREPROCESSING', DATA_PREPROCESSING)

HOME d:\Data Science Programming\Project
DATASET FOLDER d:\Data Science Programming\Project\raw_animes_dataset
ANIME DATASET 2023 d:\Data Science Programming\Project\raw_animes_dataset\anime-dataset-2023.csv
DATA PREPROCESSING d:\Data Science Programming\Project\Data Preprocessing


In [4]:
anime_clean_df = pd.read_csv(os.path.join(DATA_PREPROCESSING, 'anime-data-preprocessing.csv'))
anime_clean_df

,Name,Score,Genres,Synopsis,Type,Episodes,Producers,Studios,Source,Time per ep (Min),Rating,Rank,Popularity,Favorites,Scored By,Members,Released date,Completed date
0,Fullmetal Alchemist: Brotherhood,9.10,"Action, Adventure, Drama, Fantasy",After a horrific alchemy experiment goes wrong...,TV,64.0,"Aniplex, Square Enix, Mainichi Broadcasting Sy...",Bones,Manga,24.00,R - 17+ (violence & profanity),1,3,217606,2020030,3176556,2009-04-05,2010-07-04
1,Steins;Gate,9.07,"Drama, Sci-Fi, Suspense",Eccentric scientist Rintarou Okabe has a never...,TV,24.0,"Frontier Works, Media Factory, Kadokawa Shoten...",White Fox,Visual novel,24.00,PG-13 - Teens 13 or older,2,13,182964,1336233,2440369,2011-04-06,2011-09-14
2,Bleach: Sennen Kessen-hen,9.07,"Action, Adventure, Fantasy",Substitute Soul Reaper Ichigo Kurosaki spends ...,TV,13.0,"TV Tokyo, Aniplex, Dentsu, Shueisha",Pierrot,Manga,24.00,R - 17+ (violence & profanity),3,464,17999,213872,445198,2022-10-11,2022-12-27
3,Gintama°,9.06,"Action, Comedy, Sci-Fi","Gintoki, Shinpachi, and Kagura return as the f...",TV,51.0,"TV Tokyo, Aniplex, Dentsu",Bandai Namco Pictures,Manga,24.00,PG-13 - Teens 13 or older,4,331,15947,237957,595767,2015-04-08,2016-03-30
4,Shingeki no Kyojin Season 3 Part 2,9.05,"Action, Drama",Seeking to restore humanity's diminishing hope...,TV,10.0,"Production I.G, Dentsu, Mainichi Broadcasting ...",Wit Studio,Manga,23.00,R - 17+ (violence & profanity),5,24,55245,1471825,2104016,2019-04-29,2019-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13829,Kokuhaku,2.30,Horror,"Beside a certain building, a girl appears to m...",ONA,1.0,UNKNOWN,UNKNOWN,Original,0.67,R - 17+ (violence & profanity),13830,7105,17,4904,6697,2015-08-27,2015-08-27
13830,Hametsu no Mars,2.22,"Horror, Sci-Fi",Several months after a probe returning from Ma...,OVA,1.0,"Idea Factory, King Records, Design Factory",WAO World,Visual novel,19.00,R - 17+ (violence & profanity),13831,2512,295,47630,65622,2005-07-06,2005-07-06
13831,Tsui no Sora,2.22,Hentai,"After the sudden death of a student, mysteriou...",OVA,1.0,Obtain Future,UNKNOWN,Visual novel,23.00,Rx - Hentai,13832,7563,24,3436,5713,2002-08-10,2002-08-10
13832,Utsu Musume Sayuri,1.98,"Avant Garde, Comedy",Sayuri is a curious creature who lives on her ...,OVA,1.0,UNKNOWN,UNKNOWN,Original,3.00,R+ - Mild Nudity,13833,4492,50,15873,20789,2003-01-01,2003-01-01


### Check UNKNOWN in RATING column

In [5]:
unknown_rating_count = anime_clean_df[anime_clean_df['Rating']=='UNKNOWN']
unknown_rating_count

,Name,Score,Genres,Synopsis,Type,Episodes,Producers,Studios,Source,Time per ep (Min),Rating,Rank,Popularity,Favorites,Scored By,Members,Released date,Completed date


In [6]:
anime_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13834 entries, 0 to 13833
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               13834 non-null  object 
 1   Score              13834 non-null  float64
 2   Genres             13834 non-null  object 
 3   Synopsis           13834 non-null  object 
 4   Type               13834 non-null  object 
 5   Episodes           13834 non-null  float64
 6   Producers          13834 non-null  object 
 7   Studios            13834 non-null  object 
 8   Source             13834 non-null  object 
 9   Time per ep (Min)  13834 non-null  float64
 10  Rating             13834 non-null  object 
 11  Rank               13834 non-null  int64  
 12  Popularity         13834 non-null  int64  
 13  Favorites          13834 non-null  int64  
 14  Scored By          13834 non-null  int64  
 15  Members            13834 non-null  int64  
 16  Released date      138

In [7]:
anime_clean_df.isnull().sum()

Name                 0
Score                0
Genres               0
Synopsis             0
Type                 0
Episodes             0
Producers            0
Studios              0
Source               0
Time per ep (Min)    0
Rating               0
Rank                 0
Popularity           0
Favorites            0
Scored By            0
Members              0
Released date        0
Completed date       0
dtype: int64

### There are quite a few unnecessary columns. We will drop them.

In [8]:
anime_clean_df.drop(columns=['Name', 'Score', 'Episodes', 'Producers', 'Time per ep (Min)', 'Rank', 'Released date', 'Completed date'], inplace=True)
anime_clean_df.keys()

Index(['Genres', 'Synopsis', 'Type', 'Studios', 'Source', 'Rating',
       'Popularity', 'Favorites', 'Scored By', 'Members'],
      dtype='object')

### Handle UNKNOWN **Studios**, **Source**

In [9]:
unknown_studios_count = anime_clean_df[anime_clean_df['Studios']=='UNKNOWN'].shape[0]
unknown_studios_ratio = unknown_studios_count/anime_clean_df.shape[0] * 100

print('Unknown studios ratio is: ', unknown_studios_ratio)

Unknown studios ratio is:  16.95099031371982


In [10]:
anime_clean_df = anime_clean_df[anime_clean_df['Studios'] != 'UNKNOWN']
anime_clean_df

,Genres,Synopsis,Type,Studios,Source,Rating,Popularity,Favorites,Scored By,Members
0,"Action, Adventure, Drama, Fantasy",After a horrific alchemy experiment goes wrong...,TV,Bones,Manga,R - 17+ (violence & profanity),3,217606,2020030,3176556
1,"Drama, Sci-Fi, Suspense",Eccentric scientist Rintarou Okabe has a never...,TV,White Fox,Visual novel,PG-13 - Teens 13 or older,13,182964,1336233,2440369
2,"Action, Adventure, Fantasy",Substitute Soul Reaper Ichigo Kurosaki spends ...,TV,Pierrot,Manga,R - 17+ (violence & profanity),464,17999,213872,445198
3,"Action, Comedy, Sci-Fi","Gintoki, Shinpachi, and Kagura return as the f...",TV,Bandai Namco Pictures,Manga,PG-13 - Teens 13 or older,331,15947,237957,595767
4,"Action, Drama",Seeking to restore humanity's diminishing hope...,TV,Wit Studio,Manga,R - 17+ (violence & profanity),24,55245,1471825,2104016
...,...,...,...,...,...,...,...,...,...,...
13818,"Action, Sci-Fi","Based on a 1980 manga by Mikuriya Satomi, this...",OVA,Filmlink International,Manga,PG-13 - Teens 13 or older,8299,13,2475,4570
13822,"Action, Sci-Fi","After being hit by a truck in the year 2014, A...",TV,Visual Flight,Manga,PG-13 - Teens 13 or older,2234,294,41926,79139
13827,"Action, Comedy, Ecchi",Computer-animated gag shorts about two female ...,OVA,Production I.G,Original,R+ - Mild Nudity,6927,40,4633,7176
13830,"Horror, Sci-Fi",Several months after a probe returning from Ma...,OVA,WAO World,Visual novel,R - 17+ (violence & profanity),2512,295,47630,65622


In [11]:
unknown_source_count = anime_clean_df[anime_clean_df['Source']=='Unknown'].shape[0]
unknown_source_ratio = unknown_source_count/anime_clean_df.shape[0] * 100

print('Unknown source ratio is: ', unknown_source_ratio)

Unknown source ratio is:  8.068587344416398


In [12]:
anime_clean_df = anime_clean_df[anime_clean_df['Source'] != 'Unknown']
anime_clean_df

,Genres,Synopsis,Type,Studios,Source,Rating,Popularity,Favorites,Scored By,Members
0,"Action, Adventure, Drama, Fantasy",After a horrific alchemy experiment goes wrong...,TV,Bones,Manga,R - 17+ (violence & profanity),3,217606,2020030,3176556
1,"Drama, Sci-Fi, Suspense",Eccentric scientist Rintarou Okabe has a never...,TV,White Fox,Visual novel,PG-13 - Teens 13 or older,13,182964,1336233,2440369
2,"Action, Adventure, Fantasy",Substitute Soul Reaper Ichigo Kurosaki spends ...,TV,Pierrot,Manga,R - 17+ (violence & profanity),464,17999,213872,445198
3,"Action, Comedy, Sci-Fi","Gintoki, Shinpachi, and Kagura return as the f...",TV,Bandai Namco Pictures,Manga,PG-13 - Teens 13 or older,331,15947,237957,595767
4,"Action, Drama",Seeking to restore humanity's diminishing hope...,TV,Wit Studio,Manga,R - 17+ (violence & profanity),24,55245,1471825,2104016
...,...,...,...,...,...,...,...,...,...,...
13818,"Action, Sci-Fi","Based on a 1980 manga by Mikuriya Satomi, this...",OVA,Filmlink International,Manga,PG-13 - Teens 13 or older,8299,13,2475,4570
13822,"Action, Sci-Fi","After being hit by a truck in the year 2014, A...",TV,Visual Flight,Manga,PG-13 - Teens 13 or older,2234,294,41926,79139
13827,"Action, Comedy, Ecchi",Computer-animated gag shorts about two female ...,OVA,Production I.G,Original,R+ - Mild Nudity,6927,40,4633,7176
13830,"Horror, Sci-Fi",Several months after a probe returning from Ma...,OVA,WAO World,Visual novel,R - 17+ (violence & profanity),2512,295,47630,65622


### Handle 'Synopsis' columns by TFIDF

In [13]:
anime_clean_df['Synopsis']

0        After a horrific alchemy experiment goes wrong...
1        Eccentric scientist Rintarou Okabe has a never...
2        Substitute Soul Reaper Ichigo Kurosaki spends ...
3        Gintoki, Shinpachi, and Kagura return as the f...
4        Seeking to restore humanity's diminishing hope...
                               ...                        
13818    Based on a 1980 manga by Mikuriya Satomi, this...
13822    After being hit by a truck in the year 2014, A...
13827    Computer-animated gag shorts about two female ...
13830    Several months after a probe returning from Ma...
13833    When a mysterious entity suddenly appears in t...
Name: Synopsis, Length: 10562, dtype: object

In [14]:
anime_clean_df[anime_clean_df['Synopsis'] == 'No description available for this anime.']

,Genres,Synopsis,Type,Studios,Source,Rating,Popularity,Favorites,Scored By,Members
923,"Comedy, Slice of Life",No description available for this anime.,Special,Shaft,4-koma manga,PG-13 - Teens 13 or older,5179,10,7278,14601
956,Comedy,No description available for this anime.,Movie,GoHands,4-koma manga,R - 17+ (violence & profanity),3813,49,8149,30035
1438,"Comedy, Romance",No description available for this anime.,OVA,feel.,Light novel,PG-13 - Teens 13 or older,2005,489,10462,94326
1476,Hentai,No description available for this anime.,OVA,T-Rex,Manga,Rx - Hentai,7212,75,2929,6456
1623,"Drama, Fantasy, Slice of Life",No description available for this anime.,Movie,Studio Gokumi,Light novel,PG-13 - Teens 13 or older,5182,11,5270,14555
...,...,...,...,...,...,...,...,...,...,...
13037,Hentai,No description available for this anime.,OVA,Studio Jam,Visual novel,Rx - Hentai,12692,0,247,981
13062,Comedy,No description available for this anime.,Music,Toei Animation,Picture book,PG - Children,15367,1,211,450
13239,Hentai,No description available for this anime.,OVA,Image House,Visual novel,Rx - Hentai,13395,0,202,797
13283,"Supernatural, Hentai",No description available for this anime.,OVA,Studio 9 Maiami,Visual novel,Rx - Hentai,9118,2,1096,3379


In [15]:
filtering = anime_clean_df[anime_clean_df['Synopsis'] == "No description available for this anime."]
anime_clean_df.drop(filtering.index,inplace=True)
anime_clean_df.reset_index(drop=True, inplace=True)
anime_clean_df

C:\Users\MINH HUNG\AppData\Local\Temp\ipykernel_16800\345881173.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime_clean_df.drop(filtering.index,inplace=True)


,Genres,Synopsis,Type,Studios,Source,Rating,Popularity,Favorites,Scored By,Members
0,"Action, Adventure, Drama, Fantasy",After a horrific alchemy experiment goes wrong...,TV,Bones,Manga,R - 17+ (violence & profanity),3,217606,2020030,3176556
1,"Drama, Sci-Fi, Suspense",Eccentric scientist Rintarou Okabe has a never...,TV,White Fox,Visual novel,PG-13 - Teens 13 or older,13,182964,1336233,2440369
2,"Action, Adventure, Fantasy",Substitute Soul Reaper Ichigo Kurosaki spends ...,TV,Pierrot,Manga,R - 17+ (violence & profanity),464,17999,213872,445198
3,"Action, Comedy, Sci-Fi","Gintoki, Shinpachi, and Kagura return as the f...",TV,Bandai Namco Pictures,Manga,PG-13 - Teens 13 or older,331,15947,237957,595767
4,"Action, Drama",Seeking to restore humanity's diminishing hope...,TV,Wit Studio,Manga,R - 17+ (violence & profanity),24,55245,1471825,2104016
...,...,...,...,...,...,...,...,...,...,...
10274,"Action, Sci-Fi","Based on a 1980 manga by Mikuriya Satomi, this...",OVA,Filmlink International,Manga,PG-13 - Teens 13 or older,8299,13,2475,4570
10275,"Action, Sci-Fi","After being hit by a truck in the year 2014, A...",TV,Visual Flight,Manga,PG-13 - Teens 13 or older,2234,294,41926,79139
10276,"Action, Comedy, Ecchi",Computer-animated gag shorts about two female ...,OVA,Production I.G,Original,R+ - Mild Nudity,6927,40,4633,7176
10277,"Horror, Sci-Fi",Several months after a probe returning from Ma...,OVA,WAO World,Visual novel,R - 17+ (violence & profanity),2512,295,47630,65622


In [16]:
df_values = pd.DataFrame(anime_clean_df['Synopsis'])
df_values['Synopsis'] = df_values['Synopsis'].str.replace(r'[^\w\s]', "")
df_values

,Synopsis
0,After a horrific alchemy experiment goes wrong...
1,Eccentric scientist Rintarou Okabe has a never...
2,Substitute Soul Reaper Ichigo Kurosaki spends ...
3,"Gintoki, Shinpachi, and Kagura return as the f..."
4,Seeking to restore humanity's diminishing hope...
...,...
10274,"Based on a 1980 manga by Mikuriya Satomi, this..."
10275,"After being hit by a truck in the year 2014, A..."
10276,Computer-animated gag shorts about two female ...
10277,Several months after a probe returning from Ma...


In [17]:
df_values['Synopsis'] = df_values['Synopsis'].apply(word_tokenize)

In [18]:
eng_stopwords = stopwords.words('english') 
df_values['Synopsis'] = df_values['Synopsis'].dropna().apply(lambda words: [word.lower() for word in words if word.lower() not in eng_stopwords])
df_values['Synopsis'] = df_values['Synopsis'].apply(lambda x: [word.strip(string.punctuation) for word in x])
df_values['Synopsis'] = df_values['Synopsis'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
df_values['Synopsis'] = df_values['Synopsis'].apply(lambda x: [word for word in x if len(word) > 1])
df_values['Synopsis']

0        [horrific, alchemy, experiment, go, wrong, elr...
1        [eccentric, scientist, rintarou, okabe, never-...
2        [substitute, soul, reaper, ichigo, kurosaki, s...
3        [gintoki, shinpachi, kagura, return, fun-lovin...
4        [seeking, restore, humanity, diminishing, hope...
                               ...                        
10274    [based, 1980, manga, mikuriya, satomi, adaptat...
10275    [hit, truck, year, 2014, akira, natsume, awake...
10276    [computer-animated, gag, short, two, female, a...
10277    [several, month, probe, returning, mar, burn, ...
10278    [mysterious, entity, suddenly, appears, center...
Name: Synopsis, Length: 10279, dtype: object

In [19]:
df_values['Synopsis'].explode().value_counts()[:30]

Synopsis
source     3788
one        3271
world      3042
school     2912
girl       2827
life       2773
new        2379
however    2293
friend     2186
find       1967
day        1902
year       1819
time       1608
two        1563
student    1547
ann        1448
take       1426
high       1346
young      1317
power      1278
first      1267
story      1258
human      1249
must       1193
way        1174
love       1160
make       1146
begin      1144
get        1141
episode    1139
Name: count, dtype: int64

In [20]:
tfidfvectorsizer = TfidfVectorizer(max_features=500,stop_words='english',ngram_range=(1,3))
word_array = tfidfvectorsizer.fit_transform(anime_clean_df['Synopsis'])
print(word_array.toarray()[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.19312941 0.
 0.         0.         0.42099179 0.         0.19441996 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.21996144 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [21]:
tfidf_values = pd.DataFrame(word_array.toarray(), columns=tfidfvectorsizer.get_feature_names_out())
print(tfidf_values)

       abilities  ability      able  academy  accident    action  actually  \
0            0.0      0.0  0.000000      0.0   0.00000  0.000000       0.0   
1            0.0      0.0  0.172385      0.0   0.00000  0.000000       0.0   
2            0.0      0.0  0.000000      0.0   0.00000  0.000000       0.0   
3            0.0      0.0  0.000000      0.0   0.00000  0.000000       0.0   
4            0.0      0.0  0.000000      0.0   0.00000  0.000000       0.0   
...          ...      ...       ...      ...       ...       ...       ...   
10274        0.0      0.0  0.000000      0.0   0.00000  0.000000       0.0   
10275        0.0      0.0  0.000000      0.0   0.15757  0.159509       0.0   
10276        0.0      0.0  0.000000      0.0   0.00000  0.000000       0.0   
10277        0.0      0.0  0.000000      0.0   0.00000  0.000000       0.0   
10278        0.0      0.0  0.000000      0.0   0.00000  0.000000       0.0   

       adventure  adventures  age  ...    works     world     y

In [22]:
anime_clean_df = pd.concat([anime_clean_df.reset_index(),tfidf_values],axis=1)
anime_clean_df

,index,Genres,Synopsis,Type,Studios,Source,Rating,Popularity,Favorites,Scored By,...,works,world,year,year old,years,years ago,years later,young,young girl,younger
0,0,"Action, Adventure, Drama, Fantasy",After a horrific alchemy experiment goes wrong...,TV,Bones,Manga,R - 17+ (violence & profanity),3,217606,2020030,...,0.00000,0.112296,0.00000,0.0,0.141547,0.0,0.0,0.000000,0.0,0.0
1,1,"Drama, Sci-Fi, Suspense",Eccentric scientist Rintarou Okabe has a never...,TV,White Fox,Visual novel,PG-13 - Teens 13 or older,13,182964,1336233,...,0.20618,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
2,2,"Action, Adventure, Fantasy",Substitute Soul Reaper Ichigo Kurosaki spends ...,TV,Pierrot,Manga,R - 17+ (violence & profanity),464,17999,213872,...,0.00000,0.142401,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
3,3,"Action, Comedy, Sci-Fi","Gintoki, Shinpachi, and Kagura return as the f...",TV,Bandai Namco Pictures,Manga,PG-13 - Teens 13 or older,331,15947,237957,...,0.00000,0.101282,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4,4,"Action, Drama",Seeking to restore humanity's diminishing hope...,TV,Wit Studio,Manga,R - 17+ (violence & profanity),24,55245,1471825,...,0.00000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10274,10274,"Action, Sci-Fi","Based on a 1980 manga by Mikuriya Satomi, this...",OVA,Filmlink International,Manga,PG-13 - Teens 13 or older,8299,13,2475,...,0.00000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
10275,10275,"Action, Sci-Fi","After being hit by a truck in the year 2014, A...",TV,Visual Flight,Manga,PG-13 - Teens 13 or older,2234,294,41926,...,0.00000,0.000000,0.20232,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
10276,10276,"Action, Comedy, Ecchi",Computer-animated gag shorts about two female ...,OVA,Production I.G,Original,R+ - Mild Nudity,6927,40,4633,...,0.00000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
10277,10277,"Horror, Sci-Fi",Several months after a probe returning from Ma...,OVA,WAO World,Visual novel,R - 17+ (violence & profanity),2512,295,47630,...,0.00000,0.090841,0.00000,0.0,0.000000,0.0,0.0,0.109556,0.0,0.0


In [23]:
anime_clean_df.drop(['Synopsis'],axis=1,inplace=True)

In [24]:
studios_count=anime_clean_df["Studios"].value_counts()
studios_count

Studios
Toei Animation                   515
Sunrise                          389
J.C.Staff                        327
Madhouse                         296
Studio Deen                      250
                                ... 
KOO-KI                             1
Moss Design Unit                   1
Filmlink International, Bebow      1
GAV Video                          1
Mippei Eigeki Kiryuukan            1
Name: count, Length: 1228, dtype: int64

### Converting Categorical Variables

we are going to convert the tags column into a list of tags so that we could binary classify it(one hot encode)

In [25]:
anime_clean_df["Genres"]=anime_clean_df["Genres"].apply(lambda x:x.split(',') if pd.isna(x)!=True else "")

In [26]:
mlb = MultiLabelBinarizer()

anime_clean_df = pd.concat([anime_clean_df,pd.DataFrame(mlb.fit_transform(anime_clean_df["Genres"]),columns=mlb.classes_, index=anime_clean_df.index)],axis=1)

In [27]:
anime_clean_df

,index,Genres,Type,Studios,Source,Rating,Popularity,Favorites,Scored By,Members,...,Gourmet,Hentai,Horror,Mystery,Romance,Sci-Fi,Slice of Life,Sports,Supernatural,Suspense
0,0,"[Action, Adventure, Drama, Fantasy]",TV,Bones,Manga,R - 17+ (violence & profanity),3,217606,2020030,3176556,...,0,0,0,0,0,0,0,0,0,0
1,1,"[Drama, Sci-Fi, Suspense]",TV,White Fox,Visual novel,PG-13 - Teens 13 or older,13,182964,1336233,2440369,...,0,0,0,0,0,0,0,0,0,0
2,2,"[Action, Adventure, Fantasy]",TV,Pierrot,Manga,R - 17+ (violence & profanity),464,17999,213872,445198,...,0,0,0,0,0,0,0,0,0,0
3,3,"[Action, Comedy, Sci-Fi]",TV,Bandai Namco Pictures,Manga,PG-13 - Teens 13 or older,331,15947,237957,595767,...,0,0,0,0,0,0,0,0,0,0
4,4,"[Action, Drama]",TV,Wit Studio,Manga,R - 17+ (violence & profanity),24,55245,1471825,2104016,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10274,10274,"[Action, Sci-Fi]",OVA,Filmlink International,Manga,PG-13 - Teens 13 or older,8299,13,2475,4570,...,0,0,0,0,0,0,0,0,0,0
10275,10275,"[Action, Sci-Fi]",TV,Visual Flight,Manga,PG-13 - Teens 13 or older,2234,294,41926,79139,...,0,0,0,0,0,0,0,0,0,0
10276,10276,"[Action, Comedy, Ecchi]",OVA,Production I.G,Original,R+ - Mild Nudity,6927,40,4633,7176,...,0,0,0,0,0,0,0,0,0,0
10277,10277,"[Horror, Sci-Fi]",OVA,WAO World,Visual novel,R - 17+ (violence & profanity),2512,295,47630,65622,...,0,0,1,0,0,0,0,0,0,0


we also create binary columns for Type and Studio

In [28]:
anime_clean_df = pd.get_dummies(anime_clean_df,columns = ['Type', 'Studios', 'Source'])
anime_clean_df

,index,Genres,Rating,Popularity,Favorites,Scored By,Members,abilities,ability,able,...,Source_Mixed media,Source_Music,Source_Novel,Source_Original,Source_Other,Source_Picture book,Source_Radio,Source_Visual novel,Source_Web manga,Source_Web novel
0,0,"[Action, Adventure, Drama, Fantasy]",R - 17+ (violence & profanity),3,217606,2020030,3176556,0.0,0.0,0.000000,...,False,False,False,False,False,False,False,False,False,False
1,1,"[Drama, Sci-Fi, Suspense]",PG-13 - Teens 13 or older,13,182964,1336233,2440369,0.0,0.0,0.172385,...,False,False,False,False,False,False,False,True,False,False
2,2,"[Action, Adventure, Fantasy]",R - 17+ (violence & profanity),464,17999,213872,445198,0.0,0.0,0.000000,...,False,False,False,False,False,False,False,False,False,False
3,3,"[Action, Comedy, Sci-Fi]",PG-13 - Teens 13 or older,331,15947,237957,595767,0.0,0.0,0.000000,...,False,False,False,False,False,False,False,False,False,False
4,4,"[Action, Drama]",R - 17+ (violence & profanity),24,55245,1471825,2104016,0.0,0.0,0.000000,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10274,10274,"[Action, Sci-Fi]",PG-13 - Teens 13 or older,8299,13,2475,4570,0.0,0.0,0.000000,...,False,False,False,False,False,False,False,False,False,False
10275,10275,"[Action, Sci-Fi]",PG-13 - Teens 13 or older,2234,294,41926,79139,0.0,0.0,0.000000,...,False,False,False,False,False,False,False,False,False,False
10276,10276,"[Action, Comedy, Ecchi]",R+ - Mild Nudity,6927,40,4633,7176,0.0,0.0,0.000000,...,False,False,False,True,False,False,False,False,False,False
10277,10277,"[Horror, Sci-Fi]",R - 17+ (violence & profanity),2512,295,47630,65622,0.0,0.0,0.000000,...,False,False,False,False,False,False,False,True,False,False


In [29]:
genres = pd.Series([x for item in anime_clean_df["Genres"] for x in item]).value_counts()
genres.head(20)

Action            3318
Comedy            2606
 Fantasy          2113
 Sci-Fi           1826
 Romance          1439
 Comedy           1272
 Adventure        1121
 Drama            1104
 Supernatural     1004
Adventure          921
Hentai             859
Drama              735
 Ecchi             656
 Slice of Life     628
 Mystery           548
Fantasy            399
Slice of Life      244
 Horror            242
Sports             229
 Sports            217
Name: count, dtype: int64

In [30]:
genres_exploded = anime_clean_df.explode('Genres')
genres_dummies = pd.get_dummies(genres_exploded['Genres'])
genres_final = pd.concat([genres_exploded['index'], genres_dummies], axis=1)
genres_final = genres_final.groupby('index').sum()

anime_clean_df = pd.concat([anime_clean_df, genres_final], axis=1)
anime_clean_df

,index,Genres,Rating,Popularity,Favorites,Scored By,Members,abilities,ability,able,...,Gourmet,Hentai,Horror,Mystery,Romance,Sci-Fi,Slice of Life,Sports,Supernatural,Suspense
0,0,"[Action, Adventure, Drama, Fantasy]",R - 17+ (violence & profanity),3,217606,2020030,3176556,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,1,"[Drama, Sci-Fi, Suspense]",PG-13 - Teens 13 or older,13,182964,1336233,2440369,0.0,0.0,0.172385,...,0,0,0,0,0,0,0,0,0,0
2,2,"[Action, Adventure, Fantasy]",R - 17+ (violence & profanity),464,17999,213872,445198,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,3,"[Action, Comedy, Sci-Fi]",PG-13 - Teens 13 or older,331,15947,237957,595767,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,4,"[Action, Drama]",R - 17+ (violence & profanity),24,55245,1471825,2104016,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10274,10274,"[Action, Sci-Fi]",PG-13 - Teens 13 or older,8299,13,2475,4570,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
10275,10275,"[Action, Sci-Fi]",PG-13 - Teens 13 or older,2234,294,41926,79139,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
10276,10276,"[Action, Comedy, Ecchi]",R+ - Mild Nudity,6927,40,4633,7176,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
10277,10277,"[Horror, Sci-Fi]",R - 17+ (violence & profanity),2512,295,47630,65622,0.0,0.0,0.000000,...,0,0,1,0,0,0,0,0,0,0


Scaling the columns to avoid adverse effects.

In [31]:
scaler = preprocessing.MinMaxScaler()
anime_clean_df[["Popularity"]] = scaler.fit_transform(anime_clean_df[["Popularity"]])
anime_clean_df[["Favorites"]] = scaler.fit_transform(anime_clean_df[["Favorites"]])
anime_clean_df[["Scored By"]] = scaler.fit_transform(anime_clean_df[["Scored By"]])
anime_clean_df[["Members"]] = scaler.fit_transform(anime_clean_df[["Members"]])

### Building a Model

Next we split our data for training and testing.

In [32]:
anime_clean_df.keys()

Index(['index', 'Genres', 'Rating', 'Popularity', 'Favorites', 'Scored By',
       'Members', 'abilities', 'ability', 'able',
       ...
       'Gourmet', 'Hentai', 'Horror', 'Mystery', 'Romance', 'Sci-Fi',
       'Slice of Life', 'Sports', 'Supernatural', 'Suspense'],
      dtype='object', length=1839)

In [33]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Convert 'Rating' to numerical values
anime_clean_df['Rating_encoded'] = label_encoder.fit_transform(anime_clean_df['Rating'])

# Get the mapping of string labels to numeric values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Print the mapping
print("Label Mapping:", label_mapping)

# Alternatively, you can directly access classes_ and transform methods:
print("Classes in order:", label_encoder.classes_)

Label Mapping: {'G - All Ages': np.int64(0), 'PG - Children': np.int64(1), 'PG-13 - Teens 13 or older': np.int64(2), 'R - 17+ (violence & profanity)': np.int64(3), 'R+ - Mild Nudity': np.int64(4), 'Rx - Hentai': np.int64(5)}
Classes in order: ['G - All Ages' 'PG - Children' 'PG-13 - Teens 13 or older'
 'R - 17+ (violence & profanity)' 'R+ - Mild Nudity' 'Rx - Hentai']


In [34]:
X = anime_clean_df.drop(["index","Genres","Rating"],axis=1)
y = anime_clean_df["Rating_encoded"]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)

In [35]:
X_train

,Popularity,Favorites,Scored By,Members,abilities,ability,able,academy,accident,action,...,Hentai,Horror,Mystery,Romance,Sci-Fi,Slice of Life,Sports,Supernatural,Suspense,Rating_encoded
3804,0.394528,0.000142,0.000696,0.001474,0.0,0.000000,0.0,0.245067,0.0,0.0,...,0,0,0,0,0,1,0,0,0,1
10009,0.707139,0.000000,0.000073,0.000154,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
8850,0.669932,0.000018,0.000083,0.000200,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2646,0.141740,0.001006,0.009114,0.015098,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,2
2912,0.409588,0.000018,0.000769,0.001356,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,0.172329,0.000464,0.004570,0.010544,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,2
7813,0.463731,0.000014,0.000553,0.000925,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
905,0.108807,0.000694,0.013431,0.023787,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,3
5192,0.224596,0.000230,0.003623,0.006067,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1


Lets now build our model and compare the results

In [36]:
model = ExtraTreesClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_test)
mae = mean_absolute_error(y_test, preds)
print('Mean Absolute Error:',mae)

Mean Absolute Error: 0.13959143968871596


In [37]:
r2 = r2_score(preds,y_test)
print(r2)

0.8783395511868739


In [38]:
for i in range(5):
    print('Predicted: {:.2f};\n     Real: {};\n'.format(preds[i], y_test.iloc[i]))

Predicted: 2.00;
     Real: 1;

Predicted: 2.00;
     Real: 2;

Predicted: 2.00;
     Real: 3;

Predicted: 2.00;
     Real: 2;

Predicted: 1.00;
     Real: 1;

